In [24]:
import os
import numpy as np
import librosa
import pandas as pd
from collections import defaultdict
import random
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
def extract_acousticness(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path, sr=None)

    # Extract features related to acousticness
    # 1. RMS (Root Mean Square Energy)
    rms = librosa.feature.rms(y=y).mean()

    # 2. Spectral Centroid (indicates brightness of the track)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()

    # 3. Zero Crossing Rate (indicates percussiveness or transient noise)
    zcr = librosa.feature.zero_crossing_rate(y=y).mean()

    # 4. Harmonic to Percussive Ratio (Harmonic sounds dominate in acoustic tracks)
    harmonic, percussive = librosa.effects.hpss(y)
    hpr = np.sum(np.abs(harmonic)) / np.sum(np.abs(percussive))

    # Normalize features and combine into a heuristic score
    acousticness = (1 - rms) * 0.4 + (1 / (1 + spectral_centroid)) * 0.3 + (1 - zcr) * 0.2 + hpr * 0.1

    return acousticness

In [26]:
def extract_danceability(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path, sr=None)

    # 1. Tempo (beats per minute)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    
    # 2. Rhythm Stability (Regularity)
    # We can use the autocorrelation of the onset envelope to check rhythmic stability
    autocorr = np.correlate(onset_env, onset_env, mode='full')
    rhythm_stability = np.max(autocorr)

    # 3. Beat Strength (using the onset envelope)
    beat_strength = np.mean(onset_env)

    # 4. Regularity (how consistent the beat is across the track)
    # This can be approximated using the variance in beat locations
    times = librosa.frames_to_time(np.arange(len(onset_env)), sr=sr)
    beat_frames = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)[1]
    beat_regularities = np.diff(beat_frames)
    regularity = np.std(beat_regularities)  # Less variance = more regular

    # Normalize the features to combine them into a single "danceability" score
    # Scale them to be between 0 and 1 (min-max scaling)
    danceability = (tempo / 250) * 0.4 + (rhythm_stability / np.max(autocorr)) * 0.3 + (beat_strength / np.max(onset_env)) * 0.2 + (1 / (1 + regularity)) * 0.1
    
    return danceability

In [31]:
def load_data(data_dir):

    # Initialize a dictionary to store the number of files in each genre
    genre_files = defaultdict(list)

    # Loop through each genre folder and extract features
    for genre, path in data_dir.items():
        for filename in os.listdir(path):
            if filename.endswith(".mp3"):
                file_path = os.path.join(path, filename)
                
                # Load audio file
                audio, sr = librosa.load(file_path, sr=22050)
                
                # Extract features: Spectral Centroid
                spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
                spectral_centroid_mean = np.mean(spectral_centroid.T, axis=0)
                
                # Extract features: Spectral Bandwidth
                spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)
                spectral_bandwidth_mean = np.mean(spectral_bandwidth.T, axis=0)
                
                # Extract features: tempo
                tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)
                               
                # Create a dictionary for the current file's features
                features = {
                    'spectral_centroid': spectral_centroid_mean[0],
                    'spectral_bandwidth': spectral_bandwidth_mean[0],
                    'tempo': tempo,
                    'genre': genre
                }
                
                # Append the feature dictionary to genre_files
                genre_files[genre].append(features)
    
    # Find the minimum number of files in any genre to balance the data
    min_samples = min(len(genre_files["Jazz"]), len(genre_files["Classical"]))

    # Downsample or upsample genres to match the minimum number of files
    balanced_data = genre_files["Jazz"][:min_samples] + genre_files["Classical"][:min_samples]
    
    # Shuffle the data to mix the genres
    random.shuffle(balanced_data)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(balanced_data)
    return df

In [ ]:
# Define the paths to the jazz and classical folders
data_dir = {
    "Jazz": "./Jazz",
    "Classical": "./Classical"
}

print("Loading data and extracting features...")
df = load_data(data_dir)

sns.pairplot(df, hue='genre', diag_kind='kde', plot_kws={'alpha': 0.5})
plt.suptitle("Pair Plot of Selected Features by Genre", y=1.02)
plt.show()